In [ ]:
import tifffile as tf
import numpy as np
from pathlib import Path
import pandas as pd
from webfish_tools.util import find_matching_files

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        name = "Gaussian"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "LoG"
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False,
                             add_trace = True,swap_axes=True, zmax=10):
    
    if raw_image == False:
        if len(tiff.shape) == 3:
            if add_trace == False:
                plot_2d_locs_on_2d_image(df_tiff_1, None, tiff[channel], zmax=zmax, add_trace=add_trace)
            else:
                plot_2d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff[channel],add_trace=add_trace, zmax=zmax)
        else:
        #PLot All Z's that had dot detection
        #-------------------------------------------
            for z in range(len(tiff[:,channel])):
                if add_trace == False:
                    df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                    plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
                else:
                    df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                    df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                    plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        if swap_axes==True:
            tiff = np.swapaxes(tiff,0,1)
        print("shape =", tiff.shape)
        #plot
        if len(tiff.shape)==3:
            if add_trace == False:
                plot_2d_locs_on_2d_image(df_tiff_1, None, tiff[channel], zmax=zmax, add_trace=add_trace)
            else:
                plot_2d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff[channel],add_trace=add_trace, zmax=zmax)
        else:
            for z in range(len(tiff[:,channel])):
                if add_trace == False:
                    df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                    plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
                else:
                    df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                    df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                    plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)

In [ ]:
#read in adcg dots
df = pd.read_csv("/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_detected/ADCG/HybCycle_0/MMStack_Pos0_zslice_0_channel_1.csv")

In [ ]:
df["z"]=0
df["x"] = df["x"]-1
df["y"] = df["y"]-1

In [ ]:
# hyb = 0
# ch = 1 
# df_hyb = df[(df["hyb"]==hyb) & (df["ch"] == ch)]

In [ ]:
# df_hyb

In [ ]:
hyb = 0
pos = 0
img_raw = f'/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
plot_3d_locs_on_2d_image(df,None, tiff=None, channel=0, raw_src = img_raw, 
                         raw_image = True, add_trace=False, swap_axes = False, zmax=2000)

In [ ]:
# hyb = 0
# pos = 2
# img_src = f'/groups/CaiLab/personal/Lex/raw/576_readout_screen/100621_576readout_screen/notebook_pyfiles/pre_processed_images/HybCycle_{hyb}/MMStack_Pos{pos}.tif'
# tiff = tf.imread(img_src)
# plot_3d_locs_on_2d_image(df_hyb,None, tiff=tiff, channel=1, 
#                          raw_src = img_raw, raw_image = False,add_trace=False, zmax=1000)